# Task B — Portfolio Allocation
**Climate Resilience Fund — Decision-Support Prototype**

This notebook allocates a $25M budget across districts and interventions, using the delivery feasibility predictions from Task A (`dist_int_delivery.csv`).

**Constraints:**
- Equity: ≥40% of budget to top-poverty-quintile districts
- Heat balance: ≥30% to heat-focused interventions
- Concentration: ≤$1.2M per district
- Coverage: all regions funded

**Outputs:** `portfolio_allocation.csv`, `top10_districts.csv`, `top10_codes.json`

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import json, os
import warnings
warnings.filterwarnings('ignore')

In [3]:
#Give path to the inputs files. The data provided for the case study is stored in the "data" folder under "dalberg_case_study" folder
DATA = "/content/drive/MyDrive/dalberg_case_study"

---
## 1. Load Data

Four datasets are loaded: **district profiles** (120 districts with poverty, population, infrastructure, and historical loss data), **hazard timeseries** (monthly flood/heat indices per district), **interventions catalog** (18 intervention types with cost, risk reduction, and implementation time), and **dist_int_delivery.csv** from Task A (predicted on-time delivery probability for every district x intervention pair — used here as the feasibility score).

In [4]:
districts = pd.read_csv(f"{DATA}/data/district_profile.csv")
hazard = pd.read_csv(f"{DATA}/data/hazard_timeseries.csv")
interventions = pd.read_csv(f"{DATA}/data/interventions_catalog.csv")
delivery = pd.read_csv(f"{DATA}/output/dist_int_delivery.csv")

print(f"Districts:    {districts.shape}")
print(f"Hazard:       {hazard.shape}")
print(f"Interventions: {interventions.shape}")
print(f"Delivery predictions: {delivery.shape}")
display(delivery.head())

Districts:    (120, 17)
Hazard:       (5752, 9)
Interventions: (18, 10)
Delivery predictions: (2160, 12)


,district_code,district_name,region,country,intervention_id,intervention_name,hazard_focus,cost,risk_reduction,impl_months,delivery_prob,predicted_on_time
0,D-001,District 001,West,Azuria,I01,Drainage Rehabilitation (Basic),flood,969000,0.120,13,0.042243,0
1,D-001,District 001,West,Azuria,I02,River Embankment Reinforcement (Standard),flood,1263000,0.149,14,0.195205,0
2,D-001,District 001,West,Azuria,I03,Flood Shelter Retrofit (Enhanced),flood,638000,0.240,22,0.004443,0
3,D-001,District 001,West,Azuria,I04,Watershed Reforestation (Basic),flood,459000,0.118,22,0.006153,0
4,D-001,District 001,West,Azuria,I05,Early Warning System Upgrade (Standard),both,488000,0.190,22,0.004118,0


---
## 2. Compute District Vulnerability

Each district receives a **vulnerability score (0–1)** that quantifies how exposed and at-risk it is. The score is a weighted sum of six normalised indicators:

| Component | Weight | Source |
|---|---|---|
| Poverty index | 25% | District profile |
| Population | 15% | District profile |
| Historical flood losses (USD) | 15% | District profile (capped at 99th percentile) |
| Historical heat losses (USD) | 15% | District profile (capped at 99th percentile) |
| 2024 mean heat index | 15% | Hazard timeseries (annual average) |
| 2024 mean flood index | 15% | Hazard timeseries (annual average) |

Poverty is given the highest weight (25%) because the fund prioritises vulnerable communities. Historical losses and current hazard exposure together account for 60% to capture both past damage and ongoing risk. All components are min-max normalised to [0, 1] before weighting. Districts are also split into poverty quintiles; the top quintile (Q5) is flagged as `is_top_poverty` for the equity constraint.

In [5]:
# Outlier handling
for col in ['historical_flood_loss_usd', 'historical_heat_loss_usd']:
    cap = districts[col].quantile(0.99)
    districts[col] = districts[col].clip(upper=cap)

# Normalize key columns to 0-1
for col in ['population', 'poverty_index', 'historical_flood_loss_usd', 'historical_heat_loss_usd']:
    mn, mx = districts[col].min(), districts[col].max()
    districts[f'{col}_norm'] = (districts[col] - mn) / (mx - mn + 1e-9)

# Latest hazard trends (2024)
hazard['year'] = hazard['month'].str[:4].astype(int)
hazard_2024 = hazard[hazard['year'] == 2024].groupby('district_code').agg(
    mean_heat=('heat_index', 'mean'),
    mean_flood=('flood_index', 'mean'),
    total_heatwave=('heatwave_days', 'sum'),
).reset_index()

districts = districts.merge(hazard_2024, on='district_code', how='left')
for col in ['mean_heat', 'mean_flood', 'total_heatwave']:
    mn, mx = districts[col].min(), districts[col].max()
    districts[f'{col}_norm'] = (districts[col] - mn) / (mx - mn + 1e-9)

# Vulnerability = weighted combination
districts['vulnerability'] = (
    0.25 * districts['poverty_index_norm'] +
    0.15 * districts['population_norm'] +
    0.15 * districts['historical_flood_loss_usd_norm'] +
    0.15 * districts['historical_heat_loss_usd_norm'] +
    0.15 * districts['mean_heat_norm'] +
    0.15 * districts['mean_flood_norm']
)

# Poverty quintile
districts['poverty_quintile'] = pd.qcut(districts['poverty_index'], 5, labels=[1, 2, 3, 4, 5])
districts['is_top_poverty'] = (districts['poverty_quintile'] == 5).astype(int)

print(f"Districts with vulnerability scores: {len(districts)}")
print(f"Top-poverty districts: {districts['is_top_poverty'].sum()}")
display(districts[['district_code', 'district_name', 'region', 'country',
                   'poverty_index', 'vulnerability', 'is_top_poverty']].head(10))

Districts with vulnerability scores: 120
Top-poverty districts: 24


,district_code,district_name,region,country,poverty_index,vulnerability,is_top_poverty
0,D-001,District 001,West,Azuria,0.712,0.474731,1
1,D-002,District 002,Central,Belvaria,0.545,0.535344,0
2,D-003,District 003,South,Belvaria,0.219,0.146435,0
3,D-004,District 004,South,Azuria,0.341,0.244514,0
4,D-005,District 005,South,Azuria,0.536,0.314065,0
5,D-006,District 006,North,Belvaria,0.659,0.629153,1
6,D-007,District 007,West,Cordania,0.538,0.304017,0
7,D-008,District 008,Central,Azuria,0.539,0.405781,0
8,D-009,District 009,East,Belvaria,0.659,0.486286,1
9,D-010,District 010,East,Azuria,0.683,0.298906,1


---
## 3. Build Allocation Candidates (using Task A feasibility)

Each district x intervention pair is scored with an **objective function** that measures value-per-dollar:

```
objective = vulnerability × risk_reduction × feasibility / (cost / 1M)
```

- **vulnerability** — the district score computed above (higher = more at-risk, should be prioritised)
- **risk_reduction** — expected reduction from the interventions catalog (higher = more impactful)
- **feasibility** — the `delivery_prob` from Task A's model (higher = more likely to deliver on time)
- **cost** — unit cost in USD, divided by 1M so the ratio stays in a reasonable range

Pairs whose cost exceeds the $1.2M per-district cap are filtered out. The remaining candidates are sorted by objective (descending) so the greedy allocator picks highest-value-per-dollar pairs first.

In [7]:
BUDGET = 25_000_000
EQUITY_FLOOR = 0.40
HEAT_FLOOR = 0.30
MAX_PER_DISTRICT = 1_200_000

# Merge vulnerability + poverty flags onto delivery predictions
alloc_df = delivery.merge(
    districts[['district_code', 'vulnerability', 'poverty_index', 'is_top_poverty']],
    on='district_code', how='left',
    suffixes=('', '_dist')
)

# Filter out interventions that exceed the per-district cap
alloc_df = alloc_df[alloc_df['cost'] <= MAX_PER_DISTRICT].copy()

# Use delivery_prob from Task A as feasibility score
alloc_df['feasibility'] = alloc_df['delivery_prob']

# Objective per dollar = vulnerability x risk_reduction x feasibility / (cost / 1e6)
alloc_df['objective'] = (
    alloc_df['vulnerability'] * alloc_df['risk_reduction'] *
    alloc_df['feasibility'] /
    (alloc_df['cost'] / 1e6)
)

alloc_df = alloc_df.sort_values('objective', ascending=False)
print(f"Allocation candidates: {len(alloc_df)}")
display(alloc_df[['district_code', 'intervention_name', 'cost', 'feasibility',
                   'vulnerability', 'objective']].head(10))

Allocation candidates: 1920


,district_code,intervention_name,cost,feasibility,vulnerability,objective
2061,D-115,Heat-Resilient Roofing Subsidy (Basic),344000,0.847838,0.633995,0.185946
1071,D-060,Heat-Resilient Roofing Subsidy (Basic),344000,0.910449,0.579512,0.182518
999,D-056,Heat-Resilient Roofing Subsidy (Basic),344000,0.856374,0.608210,0.180180
1683,D-094,Heat-Resilient Roofing Subsidy (Basic),344000,0.869158,0.582577,0.175162
747,D-042,Heat-Resilient Roofing Subsidy (Basic),344000,0.798152,0.617687,0.170546
891,D-050,Heat-Resilient Roofing Subsidy (Basic),344000,0.799934,0.610743,0.169006
2007,D-112,Heat-Resilient Roofing Subsidy (Basic),344000,0.756384,0.634594,0.166045
675,D-038,Heat-Resilient Roofing Subsidy (Basic),344000,0.792374,0.601830,0.164965
1845,D-103,Heat-Resilient Roofing Subsidy (Basic),344000,0.583482,0.798641,0.161201
333,D-019,Heat-Resilient Roofing Subsidy (Basic),344000,0.881840,0.502333,0.153239


---
## 4. Greedy Allocation with Constraints

The allocation uses a **constrained greedy algorithm** in two passes:

**Pass 1 — Constraint satisfaction (priority fills):**
- **1a. Equity fill:** Iterate over top-poverty-quintile (Q5) candidates by objective; keep adding until ≥40% of budget is committed to Q5 districts. This guarantees the equity floor is met before general allocation begins.
- **1b. Region coverage:** For any region still unfunded after 1a, add the highest-objective candidate from that region. This ensures every region gets at least one intervention.
- **1c. Heat balance:** If heat-focused interventions are below 30% of spend, add the best heat candidates until the floor is met.

**Pass 2 — Fill remaining budget (by objective):**
Iterate over all candidates (sorted by objective) and add any that fit within the remaining budget and per-district cap. Each district can receive multiple interventions as long as total spend stays ≤$1.2M. No district-intervention pair can be selected twice.

**Pass 3 — Minimise leftover (by cost):**
After Pass 2, a small budget gap may remain if it's smaller than the next-best candidate's cost. Pass 3 sorts remaining candidates by cost (cheapest first) and tries to fit any that fall within the gap.

**Pass 4 — Partial allocation (full budget utilisation):**
If the gap is still smaller than the cheapest available intervention, the remaining amount is allocated as partial funding to the highest-objective eligible candidate. This ensures the entire $25M budget is distributed. The partial allocation is flagged in the output for transparency.

This four-pass approach ensures hard constraints are met first (Pass 1), then maximises value-per-dollar (Pass 2), fills discrete gaps (Pass 3), and guarantees full budget utilisation (Pass 4). A greedy approach is chosen over linear programming for transparency and interpretability.

In [8]:
all_regions = set(districts['region'].unique())

def run_allocation(alloc_df, budget, equity_floor, heat_floor, max_per_district, all_regions):
    """
    Greedy two-pass allocation with constraint enforcement.
    Returns (sel_df, stats_dict) where stats_dict contains spent, equity, heat, etc.
    """
    selected = []
    spent = 0
    district_spent = {}
    regions_covered = set()
    heat_spent = 0
    poverty_spent = 0
    district_interventions = set()
    equity_target = budget * equity_floor

    def try_add(row):
        nonlocal spent, heat_spent, poverty_spent
        dc = row['district_code']
        iid = row['intervention_id']
        cost = row['cost']
        if (dc, iid) in district_interventions:
            return False
        if district_spent.get(dc, 0) + cost > max_per_district:
            return False
        if spent + cost > budget:
            return False
        selected.append(row.to_dict())
        spent += cost
        district_spent[dc] = district_spent.get(dc, 0) + cost
        regions_covered.add(row['region'])
        district_interventions.add((dc, iid))
        if row['hazard_focus'] == 'heat':
            heat_spent += cost
        if row['is_top_poverty']:
            poverty_spent += cost
        return True

    # Pass 1: Fill equity floor from top-poverty districts
    for _, row in alloc_df[alloc_df['is_top_poverty'] == 1].sort_values('objective', ascending=False).iterrows():
        if poverty_spent >= equity_target:
            break
        try_add(row)

    # Pass 1b: Ensure region coverage
    for reg in all_regions - regions_covered:
        for _, row in alloc_df[alloc_df['region'] == reg].sort_values('objective', ascending=False).iterrows():
            if try_add(row):
                break

    # Pass 1c: Ensure heat floor
    for _, row in alloc_df[alloc_df['hazard_focus'] == 'heat'].sort_values('objective', ascending=False).iterrows():
        if spent > 0 and heat_spent / spent >= heat_floor:
            break
        try_add(row)

    # Pass 2: Fill remaining budget (by objective)
    for _, row in alloc_df.iterrows():
        if spent >= budget:
            break
        try_add(row)

    # Pass 3: Minimise leftover — try to fit the cheapest eligible candidate
    # into the remaining gap. Greedy-by-objective may leave a gap larger than
    # the cheapest intervention; this pass closes it.
    remaining = budget - spent
    if remaining > 0:
        for _, row in alloc_df.sort_values('cost', ascending=True).iterrows():
            if row['cost'] > remaining:
                break  # no cheaper candidate exists
            if try_add(row):
                remaining = budget - spent
                if remaining <= 0:
                    break

    # Pass 4: Partial allocation — if budget gap remains (because the cheapest
    # intervention exceeds it), allocate the remaining amount to the highest-
    # objective eligible candidate as partial funding.  This ensures the full
    # budget is distributed.
    remaining = budget - spent
    if remaining > 0:
        for _, row in alloc_df.sort_values('objective', ascending=False).iterrows():
            dc = row['district_code']
            iid = row['intervention_id']
            if (dc, iid) in district_interventions:
                continue
            if district_spent.get(dc, 0) + remaining > max_per_district:
                continue
            partial = row.to_dict()
            partial['cost'] = remaining          # fund at the remaining amount
            partial['_partial'] = True           # flag for transparency
            partial['_original_cost'] = row['cost']
            selected.append(partial)
            spent += remaining
            district_spent[dc] = district_spent.get(dc, 0) + remaining
            regions_covered.add(row['region'])
            district_interventions.add((dc, iid))
            if row['hazard_focus'] == 'heat':
                heat_spent += remaining
            if row['is_top_poverty']:
                poverty_spent += remaining
            break

    sel_df = pd.DataFrame(selected)
    stats = {
        'spent': spent,
        'poverty_spent': poverty_spent,
        'heat_spent': heat_spent,
        'district_spent': district_spent,
        'regions_covered': regions_covered,
    }
    return sel_df, stats

# Run base allocation (40% equity floor)
sel_df, base_stats = run_allocation(alloc_df, BUDGET, EQUITY_FLOOR, HEAT_FLOOR, MAX_PER_DISTRICT, all_regions)

spent = base_stats['spent']
poverty_spent = base_stats['poverty_spent']
heat_spent = base_stats['heat_spent']
district_spent = base_stats['district_spent']
regions_covered = base_stats['regions_covered']

print(f"Portfolio built: {len(sel_df)} district-intervention pairs")
print(f"Budget used: ${spent:,.0f} / ${BUDGET:,.0f}")

Portfolio built: 71 district-intervention pairs
Budget used: $25,000,000 / $25,000,000


---
## 5. Constraint Checks & Summary

After allocation, four hard constraints are verified:

1. **Equity (≥40%):** At least 40% of total spend goes to top-poverty-quintile districts
2. **Coverage (all regions):** Every region in the dataset has at least one funded intervention
3. **Concentration (≤$1.2M):** No single district receives more than 1.2M in total funding
4. **Heat balance (≥30%):** At least 30% of spend goes to heat-focused interventions

The breakdowns by region and hazard focus show how the budget is distributed geographically and across hazard types. The top-10 funded districts table identifies which districts receive the most investment — these are saved separately for use by the Task C chatbot.

In [9]:
equity_pct = poverty_spent / spent
heat_pct = heat_spent / spent
max_district = max(district_spent.values())
regions_ok = regions_covered == all_regions

checks = [
    ("Equity >= 40% to top poverty quintile", f"{equity_pct:.1%}", equity_pct >= EQUITY_FLOOR),
    ("Coverage: all regions funded", f"{len(regions_covered)}/{len(all_regions)}", regions_ok),
    ("Concentration: max per district <= $1.2M", f"${max_district:,.0f}", max_district <= MAX_PER_DISTRICT),
    ("Heat balance >= 30%", f"{heat_pct:.1%}", heat_pct >= HEAT_FLOOR),
]

print("── Constraint Checks ──")
for name, val, ok in checks:
    status = "PASS" if ok else "FAIL"
    print(f"  {'✓' if ok else '✗'} {status}  {name}: {val}")

print(f"\n── Portfolio Summary ──")
print(f"Total allocated: ${spent:,.0f} / ${BUDGET:,.0f} ({spent/BUDGET:.1%} utilisation)")
print(f"Funded pairs: {len(sel_df)}")
print(f"Unique districts: {sel_df['district_code'].nunique()}")

# Report any partial allocations
if '_partial' in sel_df.columns and sel_df['_partial'].any():
    print(f"\n── Partial Allocations ──")
    for _, r in sel_df[sel_df['_partial'] == True].iterrows():
        print(f"  {r['district_code']} — {r['intervention_name']}: "
              f"${r['cost']:,.0f} of ${r['_original_cost']:,.0f} "
              f"({r['cost']/r['_original_cost']:.0%} funded)")

── Constraint Checks ──
  ✓ PASS  Equity >= 40% to top poverty quintile: 41.5%
  ✓ PASS  Coverage: all regions funded: 5/5
  ✓ PASS  Concentration: max per district <= $1.2M: $876,000
  ✓ PASS  Heat balance >= 30%: 100.0%

── Portfolio Summary ──
Total allocated: $25,000,000 / $25,000,000 (100.0% utilisation)
Funded pairs: 71
Unique districts: 67

── Partial Allocations ──
  D-113 — Heat-Resilient Roofing Subsidy (Basic): $168,000 of $344,000 (49% funded)


In [10]:
# Breakdown by region
print("── Allocation by Region ──")
region_summary = sel_df.groupby('region').agg(
    districts=('district_code', 'nunique'),
    interventions=('intervention_id', 'count'),
    total_usd=('cost', 'sum'),
    avg_vulnerability=('vulnerability', 'mean'),
    avg_feasibility=('feasibility', 'mean'),
).round(3)
region_summary['total_usd'] = region_summary['total_usd'].apply(lambda x: f"${x:,.0f}")
display(region_summary)

# Breakdown by hazard focus
print("\n── Allocation by Hazard Focus ──")
haz_summary = sel_df.groupby('hazard_focus').agg(
    count=('intervention_id', 'count'),
    total_usd=('cost', 'sum'),
    pct=('cost', lambda x: x.sum() / spent)
).round(3)
haz_summary['total_usd'] = haz_summary['total_usd'].apply(lambda x: f"${x:,.0f}")
display(haz_summary)

── Allocation by Region ──


,districts,interventions,total_usd,avg_vulnerability,avg_feasibility
region,,,,,
Central,12,12,"$3,952,000",0.433,0.775
East,15,17,"$6,224,000",0.486,0.702
North,17,19,"$6,912,000",0.522,0.645
South,12,12,"$4,128,000",0.465,0.781
West,11,11,"$3,784,000",0.453,0.755



── Allocation by Hazard Focus ──


,count,total_usd,pct
hazard_focus,,,
heat,71,"$25,000,000",1.0


In [11]:
# Top 10 funded districts
print("── Top 10 Funded Districts ──")
top10 = sel_df.groupby(['district_code', 'district_name', 'region', 'country']).agg(
    total_usd=('cost', 'sum'),
    n_interventions=('intervention_id', 'count'),
    vulnerability=('vulnerability', 'first'),
    avg_feasibility=('feasibility', 'mean'),
    poverty_idx=('poverty_index', 'first'),
).sort_values('total_usd', ascending=False).head(10)
top10['total_usd_fmt'] = top10['total_usd'].apply(lambda x: f"${x:,.0f}")
display(top10)

top10_codes = top10.index.get_level_values('district_code').tolist()
print(f"\nTop 10 codes: {top10_codes}")

── Top 10 Funded Districts ──


,,,,total_usd,n_interventions,vulnerability,avg_feasibility,poverty_idx,total_usd_fmt
district_code,district_name,region,country,,,,,,
D-042,District 042,East,Azuria,876000,2,0.617687,0.443348,0.685,"$876,000"
D-058,District 058,North,Azuria,876000,2,0.357441,0.445563,0.732,"$876,000"
D-056,District 056,North,Belvaria,876000,2,0.608210,0.492072,0.640,"$876,000"
D-105,District 105,East,Belvaria,876000,2,0.513372,0.469942,0.657,"$876,000"
D-006,District 006,North,Belvaria,344000,1,0.629153,0.684540,0.659,"$344,000"
D-001,District 001,West,Azuria,344000,1,0.474731,0.618816,0.712,"$344,000"
D-002,District 002,Central,Belvaria,344000,1,0.535344,0.812792,0.545,"$344,000"
D-019,District 019,South,Azuria,344000,1,0.502333,0.881840,0.632,"$344,000"
D-010,District 010,East,Azuria,344000,1,0.298906,0.816909,0.683,"$344,000"



Top 10 codes: ['D-042', 'D-058', 'D-056', 'D-105', 'D-006', 'D-001', 'D-002', 'D-019', 'D-010', 'D-012']


---
## 6. Sensitivity Analysis: Equity 40% → 50%

The sensitivity test re-runs the same greedy allocation with **one change: the equity floor is raised from 40% to 50%**. This forces an additional 10% of the $25M budget (i.e., $2.5M more) into top-poverty districts. The test answers: *what is the cost of increased equity?*

By comparing the base (40%) and sensitivity (50%) portfolios side by side — total spend, number of funded pairs, average objective, average feasibility, and district composition — we can quantify the trade-off between equity and allocative efficiency. If the average objective drops, it means the additional poverty-directed spend goes to lower-value-per-dollar pairs, which is the expected price of tighter equity.

In [12]:
# Re-run allocation with tighter equity floor (50%) — reuses the same function
EQUITY_FLOOR_S = 0.50

sel_s, sens_stats = run_allocation(alloc_df, BUDGET, EQUITY_FLOOR_S, HEAT_FLOOR, MAX_PER_DISTRICT, all_regions)

spent_s = sens_stats['spent']
poverty_spent_s = sens_stats['poverty_spent']
heat_spent_s = sens_stats['heat_spent']
regions_covered_s = sens_stats['regions_covered']

equity_s = poverty_spent_s / spent_s if spent_s > 0 else 0
heat_s = heat_spent_s / spent_s if spent_s > 0 else 0

print(f"── Sensitivity: Equity Floor 40% → 50% ──")
print(f"  Budget used: ${spent_s:,.0f}")
print(f"  Equity share: {equity_s:.1%} (target: ≥50%)")
print(f"  Heat share: {heat_s:.1%}")
print(f"  Unique districts: {sel_s['district_code'].nunique()}")
print(f"  Avg objective (base): {sel_df['objective'].mean():.4f}")
print(f"  Avg objective (sensitivity): {sel_s['objective'].mean():.4f}")
obj_drop = (sel_s['objective'].mean() - sel_df['objective'].mean()) / sel_df['objective'].mean()
print(f"  Objective change: {obj_drop:+.1%}")
print(f"  → Tightening equity {'reduces' if obj_drop < 0 else 'increases'} avg value-per-dollar by {abs(obj_drop):.1%}")

── Sensitivity: Equity Floor 40% → 50% ──
  Budget used: $25,000,000
  Equity share: 50.0% (target: ≥50%)
  Heat share: 100.0%
  Unique districts: 61
  Avg objective (base): 0.1178
  Avg objective (sensitivity): 0.1130
  Objective change: -4.1%
  → Tightening equity reduces avg value-per-dollar by 4.1%


### Sensitivity Summary: What changes when the equity floor is raised from 40% to 50%?

In [13]:
# ── Side-by-side comparison: Base (40%) vs Sensitivity (50%) ──
compare = pd.DataFrame({
    'Metric': [
        'Equity floor',
        'Budget used',
        'Equity share (actual)',
        'Heat share',
        'Funded pairs',
        'Unique districts',
        'Unique regions',
        'Avg objective (value/dollar)',
        'Avg feasibility',
        'Avg vulnerability',
    ],
    'Base (40%)': [
        '40%',
        f"${spent:,.0f}",
        f"{equity_pct:.1%}",
        f"{heat_pct:.1%}",
        len(sel_df),
        sel_df['district_code'].nunique(),
        len(regions_covered),
        f"{sel_df['objective'].mean():.4f}",
        f"{sel_df['feasibility'].mean():.3f}",
        f"{sel_df['vulnerability'].mean():.3f}",
    ],
    'Sensitivity (50%)': [
        '50%',
        f"${spent_s:,.0f}",
        f"{equity_s:.1%}",
        f"{heat_s:.1%}",
        len(sel_s),
        sel_s['district_code'].nunique(),
        len(regions_covered_s),
        f"{sel_s['objective'].mean():.4f}",
        f"{sel_s['feasibility'].mean():.3f}",
        f"{sel_s['vulnerability'].mean():.3f}",
    ],
})
compare = compare.set_index('Metric')
display(compare)

# Compute changes
obj_change = (sel_s['objective'].mean() - sel_df['objective'].mean()) / sel_df['objective'].mean() * 100
feas_change = (sel_s['feasibility'].mean() - sel_df['feasibility'].mean()) / sel_df['feasibility'].mean() * 100
vuln_change = (sel_s['vulnerability'].mean() - sel_df['vulnerability'].mean()) / sel_df['vulnerability'].mean() * 100
dist_change = sel_s['district_code'].nunique() - sel_df['district_code'].nunique()
pair_change = len(sel_s) - len(sel_df)

print("\n── Summary of Changes (40% → 50% Equity Floor) ──\n")

print(f"1. Portfolio composition:")
print(f"   • Funded pairs changed by {pair_change:+d} ({len(sel_df)} → {len(sel_s)})")
print(f"   • Unique districts changed by {dist_change:+d} ({sel_df['district_code'].nunique()} → {sel_s['district_code'].nunique()})")

print(f"\n2. Efficiency trade-off:")
print(f"   • Avg objective (value-per-dollar) changed by {obj_change:+.1f}%")
if obj_change < 0:
    print(f"     → Tightening the equity constraint reduces allocative efficiency,")
    print(f"       because more budget is directed to high-poverty districts that may")
    print(f"       have lower feasibility or risk-reduction scores.")
else:
    print(f"     → The tighter equity constraint improved (or maintained) efficiency,")
    print(f"       suggesting high-poverty districts already had strong value-per-dollar.")

print(f"\n3. Feasibility impact:")
print(f"   • Avg feasibility changed by {feas_change:+.1f}%")
if feas_change < 0:
    print(f"     → Higher poverty share slightly lowers average delivery confidence.")
else:
    print(f"     → Higher poverty share maintained or improved delivery confidence.")

print(f"\n4. Equity vs efficiency:")
print(f"   • The sensitivity test demonstrates the cost of increasing equity from 40% to 50%.")
print(f"   • A {abs(obj_change):.1f}% {'decrease' if obj_change < 0 else 'increase'} in value-per-dollar is the price")
print(f"     paid for directing an additional 10% of the ${BUDGET/1e6:.0f}M budget to the poorest districts.")

# Districts unique to each portfolio
base_districts = set(sel_df['district_code'].unique())
sens_districts = set(sel_s['district_code'].unique())
gained = sens_districts - base_districts
lost = base_districts - sens_districts
if gained:
    print(f"\n5. Districts gained in sensitivity: {sorted(gained)}")
if lost:
    print(f"   Districts lost in sensitivity:   {sorted(lost)}")
if not gained and not lost:
    print(f"\n5. Same set of districts funded in both scenarios.")

,Base (40%),Sensitivity (50%)
Metric,,
Equity floor,40%,50%
Budget used,"$25,000,000","$25,000,000"
Equity share (actual),41.5%,50.0%
Heat share,100.0%,100.0%
Funded pairs,71,69
Unique districts,67,61
Unique regions,5,5
Avg objective (value/dollar),0.1178,0.1130
Avg feasibility,0.720,0.679



── Summary of Changes (40% → 50% Equity Floor) ──

1. Portfolio composition:
   • Funded pairs changed by -2 (71 → 69)
   • Unique districts changed by -6 (67 → 61)

2. Efficiency trade-off:
   • Avg objective (value-per-dollar) changed by -4.1%
     → Tightening the equity constraint reduces allocative efficiency,
       because more budget is directed to high-poverty districts that may
       have lower feasibility or risk-reduction scores.

3. Feasibility impact:
   • Avg feasibility changed by -5.7%
     → Higher poverty share slightly lowers average delivery confidence.

4. Equity vs efficiency:
   • The sensitivity test demonstrates the cost of increasing equity from 40% to 50%.
   • A 4.1% decrease in value-per-dollar is the price
     paid for directing an additional 10% of the $25M budget to the poorest districts.
   Districts lost in sensitivity:   ['D-086', 'D-090', 'D-099', 'D-107', 'D-110', 'D-113']


---
## 7. Save Outputs

Three output files are saved:

- **portfolio_allocation.csv** — the full base-case portfolio with all selected district-intervention pairs, including the `allocation` column (renamed from `cost` for clarity), vulnerability, feasibility, and objective scores
- **top10_districts.csv** — subset of the portfolio for the 10 districts with the highest total funding, used by Task C's chatbot to answer funding-related questions
- **top10_codes.json** — JSON list of the 10 district codes, used by Task C for filtering

In [14]:
# Rename 'cost' → 'allocation' for output
sel_df = sel_df.rename(columns={'cost': 'allocation'})
sel_s = sel_s.rename(columns={'cost': 'allocation'})

# Drop internal tracking columns
for df in [sel_df, sel_s]:
    for col in ['_partial', '_original_cost']:
        if col in df.columns:
            df.drop(columns=[col], inplace=True)

# Save full portfolio allocation
save_cols = ['district_code', 'district_name', 'region', 'country',
             'poverty_index', 'is_top_poverty', 'vulnerability',
             'intervention_id', 'intervention_name', 'hazard_focus',
             'allocation', 'risk_reduction', 'feasibility', 'impl_months', 'objective']
sel_df[save_cols].to_csv(f'{DATA}/output/portfolio_allocation.csv', index=False)
print(f"Saved: {DATA}/output/portfolio_allocation.csv ({len(sel_df)} rows)")

# Save top 10 districts
top10_df = sel_df[sel_df['district_code'].isin(top10_codes)]
top10_df[save_cols].to_csv(f'{DATA}/output/top10_districts.csv', index=False)
print(f"Saved: {DATA}/output/top10_districts.csv ({len(top10_df)} rows)")

# Save top 10 codes
with open(f'{DATA}/output/top10_codes.json', 'w') as f:
    json.dump(top10_codes, f)
print(f"Saved: {DATA}/output/top10_codes.json")

print(f"\nTop 10 district codes: {top10_codes}")

Saved: /content/drive/MyDrive/dalberg_case_study/output/portfolio_allocation.csv (71 rows)
Saved: /content/drive/MyDrive/dalberg_case_study/output/top10_districts.csv (14 rows)
Saved: /content/drive/MyDrive/dalberg_case_study/output/top10_codes.json

Top 10 district codes: ['D-042', 'D-058', 'D-056', 'D-105', 'D-006', 'D-001', 'D-002', 'D-019', 'D-010', 'D-012']


In [15]:
# Display portfolio preview
print("── Full Portfolio (first 30 rows) ──")
display(sel_df[save_cols].head(30))

── Full Portfolio (first 30 rows) ──


,district_code,district_name,region,country,poverty_index,is_top_poverty,vulnerability,intervention_id,intervention_name,hazard_focus,allocation,risk_reduction,feasibility,impl_months,objective
0,D-056,District 056,North,Belvaria,0.640,1,0.608210,I10,Heat-Resilient Roofing Subsidy (Basic),heat,344000,0.119,0.856374,13,0.180180
1,D-042,District 042,East,Azuria,0.685,1,0.617687,I10,Heat-Resilient Roofing Subsidy (Basic),heat,344000,0.119,0.798152,13,0.170546
2,D-103,District 103,West,Belvaria,0.664,1,0.798641,I10,Heat-Resilient Roofing Subsidy (Basic),heat,344000,0.119,0.583482,13,0.161201
3,D-006,District 006,North,Belvaria,0.659,1,0.629153,I10,Heat-Resilient Roofing Subsidy (Basic),heat,344000,0.119,0.684540,13,0.148986
4,D-105,District 105,East,Belvaria,0.657,1,0.513372,I10,Heat-Resilient Roofing Subsidy (Basic),heat,344000,0.119,0.831653,13,0.147694
5,D-009,District 009,East,Belvaria,0.659,1,0.486286,I10,Heat-Resilient Roofing Subsidy (Basic),heat,344000,0.119,0.723519,13,0.121711
6,D-100,District 100,North,Belvaria,0.854,1,0.462889,I10,Heat-Resilient Roofing Subsidy (Basic),heat,344000,0.119,0.749519,13,0.120018
7,D-064,District 064,Central,Azuria,0.759,1,0.452173,I10,Heat-Resilient Roofing Subsidy (Basic),heat,344000,0.119,0.756119,13,0.118273
8,D-080,District 080,South,Azuria,0.744,1,0.552783,I10,Heat-Resilient Roofing Subsidy (Basic),heat,344000,0.119,0.581876,13,0.111269
9,D-023,District 023,North,Belvaria,0.713,1,0.479032,I10,Heat-Resilient Roofing Subsidy (Basic),heat,344000,0.119,0.644390,13,0.106783
